<a href="https://colab.research.google.com/github/TheoLequy/HappinessHackers/blob/main/DataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

In [2]:
# !rm /content/sample_data/HappinessHackers -r

In [3]:
# Clone the github repo with all the data
%cd /content

!git clone https://github.com/TheoLequy/HappinessHackers.git

# Change to this directory
%cd HappinessHackers/data

# List the files
!ls

/content
Cloning into 'HappinessHackers'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 149 (delta 76), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (149/149), 4.89 MiB | 4.23 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/HappinessHackers/data
average-monthly-surface-temperature.csv
average-precipitation-per-year.csv
covid-excess-deaths-daily-per-100k-economist.csv
death-rate-from-suicides-gho.csv
death-rate-in-armed-conflicts.csv
economic-inequality-gini-index.csv
gdp-pcap-ppp-const2017usd.csv
happiness-cantril-ladder.csv
human-development-index.csv
human-rights-index-vdem.csv
individual-liberties-and-equality-before-the-law-index.csv
learning-adjusted-years-of-school-lays.csv
life-expectancy.csv
share-deaths-air-pollution.csv
share-of-people-who-say-that-family-is-very-important-to-them-in-life.csv
share-of-population-urban.csv
share-of-population-wi

In [6]:
# for multiple years

# Load the base data set
happiness_file_path = './happiness-cantril-ladder.csv'
data = pd.read_csv(happiness_file_path)
data = data.drop(columns='Entity')
data.columns = ['code', 'year', 'happiness']

data_directory = './'
file_paths = [os.path.join(data_directory, filename) for filename in os.listdir(data_directory) if filename.endswith('.csv')]
file_paths.remove(happiness_file_path)

for file_path in file_paths:
    try:
        if file_path == "./gdp-pcap-ppp-const2017usd.csv":
          df = pd.read_csv(file_path)
          df = df.drop(columns=['Indicator Name', 'Indicator Code', 'Unnamed: 68'])
          id_cols = ["Country Name", "Country Code"]
          df = pd.melt(df, id_vars=id_cols, var_name='year', value_name='GDPPC')
          df = df.rename(columns={"Country Name": "entity", "Country Code": "code"})
        else:
          df = pd.read_csv(file_path)

        if file_path == "./share-of-people-who-say-that-family-is-very-important-to-them-in-life.csv":
          df = df.drop(columns='Continent')

        if file_path == "./covid-excess-deaths-daily-per-100k-economist.csv":
          df['Day'] = pd.to_datetime(df['Day']).dt.year
          df = df.rename(columns={'Day': 'year'})
          df = df.drop(columns=['estimated_daily_excess_deaths_ci_95_top_per_100k', 'estimated_daily_excess_deaths_ci_95_bot_per_100k'])


        if file_path == "./average-monthly-surface-temperature.csv":
          df.columns = ['entity', 'code', 'year', 'date', 'monthly temp', 'yearly temp']
          df = df.drop(columns=['date', 'monthly temp'])
          df = df.drop_duplicates(subset=['entity', 'code', 'year'])

        # Standardize column names
        df.columns = [col.strip().lower() for col in df.columns]

        # Check if required columns are present
        required_columns = {'code', 'year'}
        if not required_columns.issubset(set(df.columns)):
          print(f"Skipping {file_path} as it lacks one or more required columns.")
          continue

        # convert years to ints
        df["year"] = df["year"].astype(int)

        if 'entity' in df.columns:
          df = df.drop(columns='entity')

        # Group by 'entity' and average the values if there are duplicates
        if df.duplicated(subset=['code', 'year']).any():
            df = df.groupby(['code', 'year']).mean().reset_index()

        data = pd.merge(data, df, on=['code', 'year'], how='left')

        print(f"Processed {file_path}: {len(df.columns)}, {df.columns}")

    except Exception as e:
        print(f"Failed to process {file_path}: {e}")
        continue


print(data.head())

if not data.empty:
    # Save the result into a new CSV file
    output_filename = '../final_dataset_raw.csv'
    data.to_csv(output_filename, index=False)

    # Group by 'code' and calculate the average for other columns
    data['covid'] = data['covid'].fillna(0)
    country_avg = data.groupby('code').transform(lambda x: x.fillna(round(x.mean(),3)))
    # Fill missing values in the original DataFrame with the calculated averages
    data_filled = data.combine_first(country_avg)

    # reorder the columns
    data_filled = data_filled[data.columns]
    # Save the result into a new CSV file
    output_filename = '../final_dataset_filled.csv'
    data_filled.to_csv(output_filename, index=False)
    print("Data processing and merging complete.")
else:
    print("No data to process.")


Processed ./gdp-pcap-ppp-const2017usd.csv: 3, Index(['code', 'year', 'gdppc'], dtype='object')
Processed ./share-of-population-urban.csv: 3, Index(['code', 'year', 'urbanization'], dtype='object')
Processed ./ti-corruption-perception-index.csv: 3, Index(['code', 'year', 'corruption'], dtype='object')
Processed ./share-of-population-with-severe-food-insecurity.csv: 3, Index(['code', 'year', 'food_insec'], dtype='object')
Processed ./tax-revenues-as-a-share-of-gdp-unu-wider.csv: 3, Index(['code', 'year', 'tax'], dtype='object')
Processed ./individual-liberties-and-equality-before-the-law-index.csv: 3, Index(['code', 'year', 'indiv_libs'], dtype='object')
Processed ./learning-adjusted-years-of-school-lays.csv: 3, Index(['code', 'year', 'school'], dtype='object')
Processed ./life-expectancy.csv: 3, Index(['code', 'year', 'life_expect'], dtype='object')
Processed ./average-monthly-surface-temperature.csv: 3, Index(['code', 'year', 'yearly temp'], dtype='object')
Processed ./human-rights-ind